In [3]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
data = pd.read_csv('/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
print(integer_encoded)
print(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print('score',score)
print('accuracy',acc)
print('Model Metrics Names',model.metrics_names)

# Saving the model
from keras.engine.saving import load_model
model.save('my_model.h5')
model1 = load_model('my_model.h5')

# Predicting the new text data on the saved model
import numpy as np
sentences = np.array(["A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"])

prediction=tokenizer.texts_to_sequences(sentences)
prediction=pad_sequences(prediction,maxlen=28)
model1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model1.predict(prediction))

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 - 19s - loss: 0.8218 - accuracy: 0.6432
Epoch 2/20
 - 18s - loss: 0.6885 - accuracy: 0.7076
Epoch 3/20
 - 18s - loss: 0.6223 - accuracy: 0.7398
Epoch 4/20
 - 18s - loss: 0.5811 - accuracy: 0.7599
Epoch 5/20
 - 17s - loss: 0.5434 - accuracy: 0.7751
Epoch 6/20
 - 17s - loss: 0.5024 - accuracy: 0.7932
Epoch 7/20
 - 17s - loss: 0.4686 - accuracy: 0.8090
Epoch 8/20
 - 17s - loss: 0.4379 - accuracy: 0.8194
Epoch 9/20
 - 17s - loss: 0.4053 - accuracy: 0.8377
Epoch 10/20
 - 17s - loss: 0.3789 - accuracy: 0.8485
Epoch 11/20
 - 17s - loss: 0.3603 - accuracy: 0.8542
Epoch 12/20
 - 17s - loss: 0.3438 - accuracy: 0.8579
Epoch 13/20
 - 17s - loss: 0.3251 - accuracy: 0.8685
Epoch 14/20
 - 17s - loss: 0.3013 - accuracy: 0.8809
Epoch 15/20
 - 17s - loss: 0.2927 - accuracy: 0.8820
Epoch 16/20
 - 17s - loss: 0.2732 - accuracy: 0.8891
Epoch 17/20
 - 17s - loss: 0.2594 - accuracy: 0.8947
Epoch 18/20
 - 17s - loss: 0.2512 - accuracy: 0.8997
Epoch 19/20
 - 18s - loss: 0.2494 - accuracy: 0.9009
Ep